# Multi-Modal Context Boundaries: Analysis Notebook

This notebook analyzes the results of experiments testing different context boundary mechanisms against multi-modal prompt injection attacks.

In [ ]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn

In [81]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import datetime
from glob import glob

# Set up plotting styles
plt.style.use('default')
sns.set_palette('muted')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
def find_latest_checkpoint(base_dir="../data"):
    """
    Find the most recent checkpoint file across all possible locations.
    
    Args:
        base_dir: Base directory to search from (default: "../data")
    
    Returns:
        tuple: (checkpoint_path, checkpoint_data) or (None, None) if no checkpoints found
    """
    # Possible checkpoint locations
    checkpoint_dirs = [
        os.path.join(base_dir, "checkpoints"),
        os.path.join(base_dir, "results", "*", "checkpoints"),
        os.path.join(base_dir, "results", "checkpoints"),
    ]
    
    all_checkpoints = []
    
    # Search all possible locations
    for pattern in checkpoint_dirs:
        if "*" in pattern:
            # Handle wildcard patterns
            dirs = glob(pattern)
            for dir_path in dirs:
                if os.path.isdir(dir_path):
                    checkpoint_files = glob(os.path.join(dir_path, "checkpoint_*.json"))
                    all_checkpoints.extend(checkpoint_files)
        else:
            # Handle direct paths
            if os.path.isdir(pattern):
                checkpoint_files = glob(os.path.join(pattern, "checkpoint_*.json"))
                all_checkpoints.extend(checkpoint_files)
    
    if not all_checkpoints:
        return None, None
    
    # Sort by modification time (most recent first)
    all_checkpoints.sort(key=os.path.getmtime, reverse=True)
    
    latest_checkpoint = all_checkpoints[0]
    
    try:
        with open(latest_checkpoint, 'r') as f:
            checkpoint_data = json.load(f)
        
        print(f"✅ Found latest checkpoint: {latest_checkpoint}")
        print(f"📅 Last modified: {datetime.datetime.fromtimestamp(os.path.getmtime(latest_checkpoint))}")
        
        return latest_checkpoint, checkpoint_data
    
    except Exception as e:
        print(f"❌ Error loading checkpoint {latest_checkpoint}: {e}")
        return None, None

def find_latest_results(base_dir="../data"):
    """
    Find the most recent results directory and load results from it.
    
    Args:
        base_dir: Base directory to search from (default: "../data")
    
    Returns:
        tuple: (results_path, results_df, metrics_df) or (None, None, None) if no results found
    """
    # Look for results directories
    results_dirs = glob(os.path.join(base_dir, "results", "experiment_*"))
    results_dirs.extend(glob(os.path.join(base_dir, "results", "run_*")))
    
    if not results_dirs:
        return None, None, None
    
    # Sort by modification time (most recent first)
    results_dirs.sort(key=os.path.getmtime, reverse=True)
    
    latest_results_dir = results_dirs[0]
    
    # Look for results files in the latest directory
    results_files = glob(os.path.join(latest_results_dir, "results_*.json"))
    results_files.extend(glob(os.path.join(latest_results_dir, "results.csv")))
    
    if results_files:
        results_files.sort(key=os.path.getmtime, reverse=True)
        latest_results_file = results_files[0]
        
        try:
            if latest_results_file.endswith('.json'):
                with open(latest_results_file, 'r') as f:
                    results_data = json.load(f)
                results_df = pd.DataFrame(results_data)
            else:  # CSV
                results_df = pd.read_csv(latest_results_file)
            
            # Try to load metrics if available
            metrics_files = glob(os.path.join(latest_results_dir, "metrics*.csv"))
            metrics_df = None
            if metrics_files:
                metrics_files.sort(key=os.path.getmtime, reverse=True)
                try:
                    metrics_df = pd.read_csv(metrics_files[0])
                except:
                    pass
            
            print(f"✅ Found latest results: {latest_results_file}")
            # Fix: Use datetime.datetime.fromtimestamp instead of datetime.fromtimestamp
            print(f"📅 Last modified: {datetime.datetime.fromtimestamp(os.path.getmtime(latest_results_file))}")
            
            return latest_results_file, results_df, metrics_df
        
        except Exception as e:
            print(f"❌ Error loading results {latest_results_file}: {e}")
    
    return None, None, None

def safe_load_json_results(file_path):
    """
    Safely load JSON results with error handling for malformed files.
    
    Args:
        file_path: Path to the JSON file
    
    Returns:
        tuple: (success, data) where success is bool and data is the loaded content or error message
    """
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return True, data
    except json.JSONDecodeError as e:
        print(f"❌ JSON decode error in {file_path}: {e}")
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()
            results = []
            for line in lines:
                if line.strip():
                    results.append(json.loads(line.strip()))
            print(f"🔧 Successfully parsed as JSONL with {len(results)} entries")
            return True, results
        except Exception as jsonl_error:
            print(f"❌ Also failed as JSONL: {jsonl_error}")
            return False, str(e)
    except Exception as e:
        print(f"❌ General error loading {file_path}: {e}")
        return False, str(e)

# Main data loading logic
print("🔍 Searching for the most recent experiment data...")
print("=" * 60)

# First, try to find checkpoint data
checkpoint_path, checkpoint_data = find_latest_checkpoint()
results_df = None
metrics_df = None

if checkpoint_data and 'results' in checkpoint_data:
    # Load from checkpoint
    try:
        results_df = pd.DataFrame(checkpoint_data['results'])
        print(f"📊 Loaded {len(results_df)} experiments from checkpoint")
        
        # Display checkpoint info
        if 'timestamp' in checkpoint_data:
            print(f"⏰ Checkpoint timestamp: {checkpoint_data['timestamp']}")
        if 'last_completed' in checkpoint_data:
            print(f"📈 Experiments completed: {checkpoint_data['last_completed']}")
        if 'config_hash' in checkpoint_data:
            print(f"🔧 Config hash: {checkpoint_data['config_hash'][:12]}...")
    except Exception as e:
        print(f"❌ Error processing checkpoint data: {e}")
        checkpoint_data = None

else:
    # Fallback: try to find results files
    print("⚠️  No valid checkpoint found, searching for results files...")
    results_path, results_df, metrics_df = find_latest_results()
    
    if results_df is not None:
        print(f"📊 Loaded {len(results_df)} experiments from results file")
    else:
        # Additional fallback: try to find any JSON files in results directories
        print("🔍 Searching for any result files in results directories...")
        
        results_dirs = glob("../data/results/run_*")
        results_dirs.extend(glob("../data/results/experiment_*"))
        
        if results_dirs:
            results_dirs.sort(key=os.path.getmtime, reverse=True)
            
            for results_dir in results_dirs[:3]:  # Check top 3 most recent
                print(f"🔍 Checking directory: {results_dir}")
                
                # Look for any JSON files
                json_files = glob(os.path.join(results_dir, "*.json"))
                json_files.extend(glob(os.path.join(results_dir, "**", "*.json")))
                
                for json_file in json_files:
                    if "checkpoint" in json_file or "results" in json_file:
                        print(f"   📄 Found: {os.path.basename(json_file)}")
                        success, data = safe_load_json_results(json_file)
                        
                        if success:
                            if isinstance(data, list) and len(data) > 0:
                                # Direct results list
                                try:
                                    results_df = pd.DataFrame(data)
                                    print(f"   ✅ Successfully loaded {len(results_df)} experiments")
                                    break
                                except Exception as e:
                                    print(f"   ❌ Error converting to DataFrame: {e}")
                            elif isinstance(data, dict) and 'results' in data:
                                # Checkpoint format
                                try:
                                    results_df = pd.DataFrame(data['results'])
                                    print(f"   ✅ Successfully loaded {len(results_df)} experiments from checkpoint format")
                                    break
                                except Exception as e:
                                    print(f"   ❌ Error converting checkpoint results to DataFrame: {e}")
                
                if results_df is not None:
                    break
        
        if results_df is None:
            print("❌ No experiment data found!")
            print("\n💡 To generate data, run one of these commands:")
            print("   python src/main.py --quick")
            print("   python src/main.py --config config/experiment.yaml")

# Display data overview if we have results
if results_df is not None and len(results_df) > 0:
    print("\n" + "=" * 60)
    print("📋 DATASET OVERVIEW")
    print("=" * 60)
    
    print(f"Total experiments: {len(results_df)}")
    
    # Check required columns
    required_cols = ['model', 'boundary', 'attack_type', 'attack_success']
    missing_cols = [col for col in required_cols if col not in results_df.columns]
    
    if missing_cols:
        print(f"⚠️  Missing columns: {missing_cols}")
        print(f"Available columns: {list(results_df.columns)}")
        
        # Try to infer missing columns or use defaults
        if 'attack_success' not in results_df.columns:
            # Check if there are any success-related columns
            success_like_cols = [col for col in results_df.columns if 'success' in col.lower() or 'result' in col.lower()]
            if success_like_cols:
                print(f"🔧 Found potential success columns: {success_like_cols}")
                results_df['attack_success'] = results_df[success_like_cols[0]]
            else:
                print("🔧 No success column found, creating dummy data for demo")
                results_df['attack_success'] = np.random.choice([0, 1], size=len(results_df))
        
        # Add missing columns with reasonable defaults
        if 'attack_subtype' not in results_df.columns:
            results_df['attack_subtype'] = 'unknown'
            
    else:
        print(f"✅ All required columns present")
        
        # Display summary stats
        print(f"\nModels tested: {list(results_df['model'].unique())}")
        print(f"Boundary types: {list(results_df['boundary'].unique())}")
        print(f"Attack types: {list(results_df['attack_type'].unique())}")
        
        # Basic success rate
        if 'attack_success' in results_df.columns:
            # Handle both boolean and numeric success indicators
            success_col = results_df['attack_success']
            if success_col.dtype == 'object':
                # Try to convert string/mixed types
                success_col = pd.to_numeric(success_col, errors='coerce')
                results_df['attack_success'] = success_col
            
            # Fill any NaN values with 0
            success_col = success_col.fillna(0)
            
            total_success = success_col.sum()
            total_experiments = len(success_col.dropna())
            success_rate = (total_success / total_experiments) * 100 if total_experiments > 0 else 0
            
            print(f"\nOverall attack success rate: {success_rate:.1f}%")
            print(f"Successful attacks: {int(total_success)}/{total_experiments}")
    
    print("\n" + "=" * 60)
    print("🚀 Ready for analysis! Continue with the cells below...")
    print("=" * 60)

else:
    print("\n❌ No valid experiment data found. Please run experiments first.")
    print("\n💡 Quick start commands:")
    print("   cd /path/to/mmcb-defense")
    print("   python src/main.py --quick")

# Display sample of the data if available
if results_df is not None and len(results_df) > 0:
    print("\n📋 Sample of loaded data:")
    print(results_df.head())
    print(f"\nData types:")
    print(results_df.dtypes)

## 1. Overview of Results

First, let's get a general overview of the experiment results.

In [ ]:
if results_df is not None:
    # Basic statistics
    total_experiments = len(results_df)
    successful_attacks = results_df['attack_success'].sum()
    success_rate = (successful_attacks / total_experiments) * 100
    
    print(f"Total experiments: {total_experiments}")
    print(f"Successful attacks: {successful_attacks} ({success_rate:.2f}%)")
    
    # Success rate by model
    model_success = results_df.groupby('model')['attack_success'].mean() * 100
    print("\nSuccess rate by model:")
    print(model_success)
    
    # Success rate by boundary type
    boundary_success = results_df.groupby('boundary')['attack_success'].mean() * 100
    print("\nSuccess rate by boundary type:")
    print(boundary_success)
    
    # Success rate by attack type
    attack_success = results_df.groupby('attack_type')['attack_success'].mean() * 100
    print("\nSuccess rate by attack type:")
    print(attack_success)
    
    # Create a summary figure
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Plot success rates by model
    sns.barplot(x=model_success.index, y=model_success.values, ax=axs[0])
    axs[0].set_title('Attack Success Rate by Model', fontsize=14)
    axs[0].set_xlabel('Model', fontsize=12)
    axs[0].set_ylabel('Success Rate (%)', fontsize=12)
    axs[0].grid(axis='y', alpha=0.3)
    axs[0].tick_params(axis='x', rotation=45)
    
    # Plot success rates by boundary
    sns.barplot(x=boundary_success.index, y=boundary_success.values, ax=axs[1])
    axs[1].set_title('Attack Success Rate by Boundary', fontsize=14)
    axs[1].set_xlabel('Boundary Type', fontsize=12)
    axs[1].set_ylabel('Success Rate (%)', fontsize=12)
    axs[1].grid(axis='y', alpha=0.3)
    
    # Plot success rates by attack type
    sns.barplot(x=attack_success.index, y=attack_success.values, ax=axs[2])
    axs[2].set_title('Attack Success Rate by Attack Type', fontsize=14)
    axs[2].set_xlabel('Attack Type', fontsize=12)
    axs[2].set_ylabel('Success Rate (%)', fontsize=12)
    axs[2].grid(axis='y', alpha=0.3)
    axs[2].tick_params(axis='x', rotation=45)
    
    # Add note that lower is better
    fig.text(0.5, 0.01, "Lower success rate indicates better protection against attacks", ha='center', fontsize=12)
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

## 2. Boundary Effectiveness Comparison

Let's analyze the effectiveness of different boundary mechanisms.

In [ ]:
if results_df is not None:
    # Create a pivot table for comprehensive comparison
    pivot_df = pd.pivot_table(
        results_df,
        values='attack_success',
        index=['model', 'attack_type'],
        columns=['boundary'],
        aggfunc=lambda x: np.mean(x) * 100  # Convert to percentage
    )
    
    # Fill any missing values
    pivot_df = pivot_df.fillna(0)
    
    # Display the pivot table
    print("Attack Success Rate (%) by Model, Attack Type, and Boundary Mechanism:")
    display(pivot_df.round(2))
    
    # Calculate the reduction in success rate compared to no boundary
    if 'none' in pivot_df.columns:
        for boundary in pivot_df.columns:
            if boundary != 'none':
                pivot_df[f'{boundary}_reduction'] = pivot_df['none'] - pivot_df[boundary]
        
        # Display the reduction
        reduction_cols = [col for col in pivot_df.columns if '_reduction' in col]
        if reduction_cols:
            print("\nReduction in Attack Success Rate (percentage points) compared to No Boundary:")
            display(pivot_df[reduction_cols].round(2))
            
            # Create heatmap of the effectiveness (reduction)
            plt.figure(figsize=(14, 8))
            sns.heatmap(pivot_df[reduction_cols], annot=True, fmt='.1f', cmap='YlGnBu')
            plt.title('Boundary Effectiveness (Reduction in Attack Success Rate)', fontsize=16)
            plt.ylabel('Model / Attack Type', fontsize=14)
            plt.xlabel('Boundary Type', fontsize=14)
            plt.xticks(rotation=45, ha='right')
            
            # Add note about the plot
            plt.figtext(0.5, 0.01, "Higher values (darker colors) indicate better protection", ha='center', fontsize=12)
            
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.show()

## 3. Cross-Modal Transfer Effectiveness

Let's analyze how well boundaries in one modality transfer to other modalities.

In [ ]:
if results_df is not None:
    print("🔍 Debugging data structure...")
    print(f"Available attack types: {results_df['attack_type'].unique()}")
    print(f"Available boundaries: {results_df['boundary'].unique()}")
    print(f"Data shape: {results_df.shape}")
    
    # Map actual attack types to modality categories for MMCB project
    modality_mapping = {
        'json': 'Structured Data',
        'csv': 'Structured Data', 
        'yaml': 'Structured Data',
        'xml': 'Structured Data',
        'python': 'Code',
        'javascript': 'Code'
    }
    
    # Add modality column to results
    results_df['modality'] = results_df['attack_type'].map(modality_mapping)
    
    # Filter out any unmapped attack types
    mapped_results = results_df.dropna(subset=['modality'])
    
    if len(mapped_results) == 0:
        print("❌ No data matches the expected attack types for MMCB")
        print("This suggests the data might be from a different experiment or have different column names.")
        print("\nActual data preview:")
        print(mapped_results.head())
    else:
        print(f"✅ Successfully mapped {len(mapped_results)} experiments to modalities")
        
        # Create visualization data
        modal_data = []
        
        # Group by modality and boundary to calculate success rates
        for modality in mapped_results['modality'].unique():
            modality_df = mapped_results[mapped_results['modality'] == modality]
            
            for boundary in modality_df['boundary'].unique():
                boundary_df = modality_df[modality_df['boundary'] == boundary]
                success_rate = boundary_df['attack_success'].mean() * 100
                
                modal_data.append({
                    'Modality': modality,
                    'Boundary': boundary,
                    'Attack Success Rate (%)': success_rate,
                    'Sample Count': len(boundary_df)
                })
        
        modal_df = pd.DataFrame(modal_data)
        
        if len(modal_df) > 0:
            print(f"📊 Generated visualization data with {len(modal_df)} points")
            
            # Create the first plot - success rates by modality and boundary
            plt.figure(figsize=(12, 8))
            ax = sns.barplot(data=modal_df, x='Modality', y='Attack Success Rate (%)', hue='Boundary')
            
            # Customize the plot
            plt.title('Boundary Effectiveness Across Modalities (MMCB)', fontsize=16)
            plt.xlabel('Modality', fontsize=14)
            plt.ylabel('Attack Success Rate (%)', fontsize=14)
            plt.legend(title='Boundary Type', fontsize=12)
            plt.grid(axis='y', alpha=0.3)
            
            # Add sample counts as text annotations
            for i, container in enumerate(ax.containers):
                ax.bar_label(container, labels=[f'n={int(modal_df.iloc[j]["Sample Count"])}' 
                                               for j in range(i, len(modal_df), len(ax.containers))],
                           fontsize=10, rotation=90)
            
            # Add note that lower is better
            plt.figtext(0.5, 0.01, "Lower success rate indicates better protection", ha='center', fontsize=12)
            
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.show()
            
            # Display the raw data
            print("\n📋 Raw Data Summary:")
            display(modal_df.round(2))
            
            # Calculate cross-modal transfer effectiveness if we have a baseline
            if 'none' in modal_df['Boundary'].unique():
                print("\n🔄 Calculating cross-modal transfer effectiveness...")
                
                # Create pivot table for effectiveness calculation
                pivot = pd.pivot_table(
                    modal_df,
                    values='Attack Success Rate (%)',
                    index=['Modality'],
                    columns=['Boundary']
                )
                
                print("Pivot table:")
                display(pivot.round(2))
                
                # Calculate relative improvement over no boundary
                effectiveness_data = {}
                baseline_col = 'none'
                
                for boundary in pivot.columns:
                    if boundary != baseline_col:
                        # Calculate percentage point reduction from baseline
                        improvement = pivot[baseline_col] - pivot[boundary]
                        effectiveness_data[f'{boundary}_effectiveness'] = improvement
                
                if effectiveness_data:
                    effectiveness_df = pd.DataFrame(effectiveness_data)
                    
                    print("\nCross-Modal Transfer Effectiveness (percentage point reduction):")
                    display(effectiveness_df.round(2))
                    
                    # Plot the effectiveness
                    plt.figure(figsize=(10, 6))
                    effectiveness_df.plot(kind='bar', ax=plt.gca())
                    
                    # Customize the plot
                    plt.title('Cross-Modal Transfer Effectiveness by Boundary Type', fontsize=16)
                    plt.xlabel('Modality', fontsize=14)
                    plt.ylabel('Attack Success Rate Reduction (percentage points)', fontsize=14)
                    plt.legend(title='Boundary Type', bbox_to_anchor=(1.05, 1), loc='upper left')
                    plt.grid(axis='y', alpha=0.3)
                    plt.xticks(rotation=45)
                    
                    # Add note about the plot
                    plt.figtext(
                        0.5, 0.01,
                        "Higher values indicate better protection improvement over no boundary",
                        ha='center',
                        fontsize=12
                    )
                    
                    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                    plt.show()
                else:
                    print("⚠️ No baseline boundary found for effectiveness calculation")
            
            else:
                print("ℹ️ No 'none' boundary baseline found - comparing boundary effectiveness directly")
                print("This is normal when experiments focus on comparing different boundary types.")
                
                # Calculate protection rates (100 - success rate)
                modal_df['Protection Rate (%)'] = 100 - modal_df['Attack Success Rate (%)']
                
                # Create a more comprehensive comparison
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
                
                # Left plot: Protection rates
                protection_pivot = pd.pivot_table(
                    modal_df,
                    values='Protection Rate (%)',
                    index=['Modality'],
                    columns=['Boundary']
                )
                
                protection_pivot.plot(kind='bar', ax=ax1)
                ax1.set_title('Protection Rates by Modality and Boundary', fontsize=14)
                ax1.set_xlabel('Modality', fontsize=12)
                ax1.set_ylabel('Protection Rate (%)', fontsize=12)
                ax1.legend(title='Boundary Type', fontsize=10)
                ax1.grid(axis='y', alpha=0.3)
                ax1.tick_params(axis='x', rotation=45)
                
                # Right plot: Relative boundary comparison (best boundary as reference)
                relative_effectiveness = {}
                best_boundary_per_modality = protection_pivot.idxmax(axis=1)
                
                for modality in protection_pivot.index:
                    best_boundary = best_boundary_per_modality[modality]
                    best_protection = protection_pivot.loc[modality, best_boundary]
                    
                    for boundary in protection_pivot.columns:
                        if boundary != best_boundary:
                            relative_diff = protection_pivot.loc[modality, boundary] - best_protection
                            if modality not in relative_effectiveness:
                                relative_effectiveness[modality] = {}
                            relative_effectiveness[modality][f'{boundary}_vs_best'] = relative_diff
                
                if relative_effectiveness:
                    rel_df = pd.DataFrame(relative_effectiveness).T
                    rel_df.plot(kind='bar', ax=ax2)
                    ax2.set_title('Boundary Performance vs Best in Each Modality', fontsize=14)
                    ax2.set_xlabel('Modality', fontsize=12)
                    ax2.set_ylabel('Protection Difference vs Best (%)', fontsize=12)
                    ax2.legend(title='Comparison', fontsize=10)
                    ax2.grid(axis='y', alpha=0.3)
                    ax2.tick_params(axis='x', rotation=45)
                    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Best performance line')
                
                plt.tight_layout()
                plt.show()
                
                # Display detailed analysis
                print("\n📊 Protection Rates by Modality and Boundary:")
                display(protection_pivot.round(2))
                
                print("\n🏆 Best Performing Boundary per Modality:")
                best_summary = []
                for modality in protection_pivot.index:
                    best_boundary = protection_pivot.loc[modality].idxmax()
                    best_rate = protection_pivot.loc[modality].max()
                    worst_boundary = protection_pivot.loc[modality].idxmin()
                    worst_rate = protection_pivot.loc[modality].min()
                    
                    best_summary.append({
                        'Modality': modality,
                        'Best Boundary': best_boundary,
                        'Best Protection (%)': best_rate,
                        'Worst Boundary': worst_boundary,
                        'Worst Protection (%)': worst_rate,
                        'Performance Gap (%)': best_rate - worst_rate
                    })
                
                best_summary_df = pd.DataFrame(best_summary)
                display(best_summary_df.round(2))
                
                print("\n📈 Cross-Modal Consistency Analysis:")
                # Check which boundary performs consistently across modalities
                boundary_ranks = {}
                for boundary in protection_pivot.columns:
                    ranks = []
                    for modality in protection_pivot.index:
                        modality_rates = protection_pivot.loc[modality].sort_values(ascending=False)
                        rank = list(modality_rates.index).index(boundary) + 1
                        ranks.append(rank)
                    
                    boundary_ranks[boundary] = {
                        'Average Rank': np.mean(ranks),
                        'Rank Std Dev': np.std(ranks),
                        'Best Modalities': sum(1 for r in ranks if r == 1),
                        'Worst Modalities': sum(1 for r in ranks if r == len(protection_pivot.columns))
                    }
                
                consistency_df = pd.DataFrame(boundary_ranks).T
                consistency_df = consistency_df.sort_values('Average Rank')
                display(consistency_df.round(3))
                
                print("\n💡 Interpretation:")
                most_consistent = consistency_df.index[0]
                print(f"• Most consistent boundary: {most_consistent} (lowest average rank)")
                print(f"• This boundary provides reliable protection across different attack modalities")
                print(f"• Lower 'Rank Std Dev' indicates more consistent performance across modalities")
        
        else:
            print("❌ No data available for visualization after processing")
else:
    print("❌ No results_df available - please run experiments first")

## 4. Implementation Complexity Analysis

Let's analyze the relationship between implementation complexity and security effectiveness.

In [ ]:
if results_df is not None:
    # Define complexity scores and estimated characteristics for each boundary type
    complexity_metrics = {
        'none': {
            'complexity_score': 0,
            'estimated_prompt_overhead': 0,  # No additional prompt content
            'implementation_hours': 0,
            'maintenance_difficulty': 1
        },
        'token': {
            'complexity_score': 2,
            'estimated_prompt_overhead': 50,  # Moderate token overhead
            'implementation_hours': 4,
            'maintenance_difficulty': 2
        },
        'semantic': {
            'complexity_score': 3,
            'estimated_prompt_overhead': 150,  # More verbose explanations
            'implementation_hours': 8,
            'maintenance_difficulty': 3
        },
        'hybrid': {
            'complexity_score': 4,
            'estimated_prompt_overhead': 200,  # Combines both approaches
            'implementation_hours': 16,
            'maintenance_difficulty': 4
        }
    }
    
    # Calculate actual performance metrics from results
    boundary_performance = results_df.groupby('boundary').agg({
        'attack_success': ['mean', 'std', 'count']
    }).round(4)
    
    boundary_performance.columns = ['success_rate', 'success_std', 'sample_count']
    boundary_performance = boundary_performance.reset_index()
    boundary_performance['protection_rate'] = (1 - boundary_performance['success_rate']) * 100
    
    # Create comprehensive complexity analysis data
    complexity_data = []
    for boundary in boundary_performance['boundary']:
        if boundary in complexity_metrics:
            metrics = complexity_metrics[boundary]
            perf = boundary_performance[boundary_performance['boundary'] == boundary].iloc[0]
            
            complexity_data.append({
                'boundary': boundary,
                'complexity_score': metrics['complexity_score'],
                'estimated_prompt_overhead': metrics['estimated_prompt_overhead'],
                'implementation_hours': metrics['implementation_hours'],
                'maintenance_difficulty': metrics['maintenance_difficulty'],
                'protection_rate': perf['protection_rate'],
                'success_rate': perf['success_rate'] * 100,
                'sample_count': perf['sample_count']
            })
    
    complexity_df = pd.DataFrame(complexity_data)
    
    if len(complexity_df) > 0:
        # Create visualization 1: Security vs Implementation Complexity
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # Plot 1: Protection Rate vs Complexity Score
        scatter1 = ax1.scatter(
            complexity_df['complexity_score'],
            complexity_df['protection_rate'],
            s=complexity_df['implementation_hours'] * 10,  # Size based on implementation time
            alpha=0.7,
            c=complexity_df['maintenance_difficulty'],
            cmap='viridis'
        )
        
        # Add labels for each point
        for i, row in complexity_df.iterrows():
            ax1.annotate(
                row['boundary'],
                (row['complexity_score'], row['protection_rate']),
                xytext=(5, 5),
                textcoords='offset points',
                fontsize=10,
                fontweight='bold'
            )
        
        ax1.set_title('Protection Rate vs Implementation Complexity', fontsize=14)
        ax1.set_xlabel('Complexity Score', fontsize=12)
        ax1.set_ylabel('Protection Rate (%)', fontsize=12)
        ax1.grid(alpha=0.3)
        
        # Plot 2: Estimated Prompt Overhead vs Protection Rate
        scatter2 = ax2.scatter(
            complexity_df['estimated_prompt_overhead'],
            complexity_df['protection_rate'],
            s=complexity_df['complexity_score'] * 50,
            alpha=0.7,
            c=complexity_df['implementation_hours'],
            cmap='plasma'
        )
        
        for i, row in complexity_df.iterrows():
            ax2.annotate(
                row['boundary'],
                (row['estimated_prompt_overhead'], row['protection_rate']),
                xytext=(5, 5),
                textcoords='offset points',
                fontsize=10,
                fontweight='bold'
            )
        
        ax2.set_title('Protection Rate vs Prompt Overhead', fontsize=14)
        ax2.set_xlabel('Estimated Prompt Overhead (characters)', fontsize=12)
        ax2.set_ylabel('Protection Rate (%)', fontsize=12)
        ax2.grid(alpha=0.3)
        
        # Plot 3: Implementation Hours vs Protection Rate
        bars3 = ax3.bar(
            complexity_df['boundary'],
            complexity_df['implementation_hours'],
            color=plt.cm.viridis(complexity_df['protection_rate'] / 100),
            alpha=0.7
        )
        
        ax3.set_title('Implementation Time by Boundary Type', fontsize=14)
        ax3.set_xlabel('Boundary Type', fontsize=12)
        ax3.set_ylabel('Implementation Hours', fontsize=12)
        ax3.tick_params(axis='x', rotation=45)
        
        # Add protection rate labels on bars
        for bar, protection in zip(bars3, complexity_df['protection_rate']):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{protection:.1f}%',
                    ha='center', va='bottom', fontweight='bold')
        
        # Plot 4: Efficiency Analysis (Protection per Complexity Unit)
        complexity_df['efficiency'] = complexity_df['protection_rate'] / (complexity_df['complexity_score'] + 0.1)  # Avoid division by zero
        
        bars4 = ax4.bar(
            complexity_df['boundary'],
            complexity_df['efficiency'],
            color=plt.cm.plasma(complexity_df['efficiency'] / complexity_df['efficiency'].max()),
            alpha=0.7
        )
        
        ax4.set_title('Protection Efficiency (Protection Rate / Complexity)', fontsize=14)
        ax4.set_xlabel('Boundary Type', fontsize=12)
        ax4.set_ylabel('Efficiency Score', fontsize=12)
        ax4.tick_params(axis='x', rotation=45)
        
        # Add efficiency scores on bars
        for bar, eff in zip(bars4, complexity_df['efficiency']):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{eff:.1f}',
                    ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Display the comprehensive complexity analysis table
        print("\n### Implementation Complexity Analysis Summary")
        display_df = complexity_df.copy()
        display_df = display_df[[
            'boundary', 'protection_rate', 'complexity_score', 
            'implementation_hours', 'estimated_prompt_overhead', 
            'maintenance_difficulty', 'efficiency'
        ]]
        display_df.columns = [
            'Boundary Type', 'Protection Rate (%)', 'Complexity Score',
            'Implementation Hours', 'Prompt Overhead (chars)', 
            'Maintenance Difficulty', 'Efficiency Score'
        ]
        display(display_df.round(2))
        
        # Calculate and display trade-off analysis
        print("\n### Security-Complexity Trade-off Analysis")
        
        # Find the most efficient boundary (highest protection per complexity unit)
        most_efficient = complexity_df.loc[complexity_df['efficiency'].idxmax()]
        
        # Find the highest protection boundary
        highest_protection = complexity_df.loc[complexity_df['protection_rate'].idxmax()]
        
        # Find the lowest complexity boundary (excluding 'none')
        non_none_df = complexity_df[complexity_df['boundary'] != 'none']
        if len(non_none_df) > 0:
            lowest_complexity = non_none_df.loc[non_none_df['complexity_score'].idxmin()]
        else:
            lowest_complexity = None
        
        trade_off_summary = []
        
        if not complexity_df.empty:
            trade_off_summary.append({
                'Metric': 'Most Efficient',
                'Boundary': most_efficient['boundary'],
                'Protection Rate (%)': f"{most_efficient['protection_rate']:.1f}",
                'Complexity Score': most_efficient['complexity_score'],
                'Efficiency Score': f"{most_efficient['efficiency']:.2f}"
            })
            
            trade_off_summary.append({
                'Metric': 'Highest Protection',
                'Boundary': highest_protection['boundary'],
                'Protection Rate (%)': f"{highest_protection['protection_rate']:.1f}",
                'Complexity Score': highest_protection['complexity_score'],
                'Efficiency Score': f"{highest_protection['efficiency']:.2f}"
            })
            
            if lowest_complexity is not None:
                trade_off_summary.append({
                    'Metric': 'Lowest Complexity',
                    'Boundary': lowest_complexity['boundary'],
                    'Protection Rate (%)': f"{lowest_complexity['protection_rate']:.1f}",
                    'Complexity Score': lowest_complexity['complexity_score'],
                    'Efficiency Score': f"{lowest_complexity['efficiency']:.2f}"
                })
        
        trade_off_df = pd.DataFrame(trade_off_summary)
        display(trade_off_df)
        
        # Generate recommendations
        print("\n### Recommendations Based on Complexity Analysis")
        
        if not complexity_df.empty:
            print(f"🏆 **Most Efficient Choice**: {most_efficient['boundary']} boundary")
            print(f"   - Provides {most_efficient['protection_rate']:.1f}% protection with complexity score {most_efficient['complexity_score']}")
            print(f"   - Best balance of security and implementation effort")
            
            print(f"\n🛡️ **Maximum Security**: {highest_protection['boundary']} boundary")
            print(f"   - Provides {highest_protection['protection_rate']:.1f}% protection")
            print(f"   - Requires {highest_protection['implementation_hours']} implementation hours")
            
            if lowest_complexity is not None:
                print(f"\n⚡ **Simplest Implementation**: {lowest_complexity['boundary']} boundary")
                print(f"   - Complexity score: {lowest_complexity['complexity_score']}")
                print(f"   - Still provides {lowest_complexity['protection_rate']:.1f}% protection")
        
        # Cost-benefit analysis
        print(f"\n### Cost-Benefit Analysis")
        if 'none' in complexity_df['boundary'].values and len(complexity_df) > 1:
            baseline_protection = complexity_df[complexity_df['boundary'] == 'none']['protection_rate'].iloc[0]
            
            print(f"**Baseline (No Boundary)**: {baseline_protection:.1f}% protection")
            print("\n**Improvement Analysis**:")
            
            for _, row in complexity_df.iterrows():
                if row['boundary'] != 'none':
                    improvement = row['protection_rate'] - baseline_protection
                    cost_per_improvement = row['implementation_hours'] / max(improvement, 0.1)
                    
                    print(f"- **{row['boundary'].title()}**: +{improvement:.1f}% protection")
                    print(f"  Cost: {row['implementation_hours']} hours ({cost_per_improvement:.1f} hours per % improvement)")
    
    else:
        print("⚠️ No boundary performance data available for complexity analysis.")
        print("This could happen if:")
        print("1. No experiments have been run yet")
        print("2. The results don't contain the expected boundary types")
        print("3. The data is corrupted or incomplete")
        
        print(f"\nAvailable boundaries in data: {list(results_df['boundary'].unique()) if 'boundary' in results_df.columns else 'No boundary column found'}")
        print(f"Available columns: {list(results_df.columns)}")

else:
    print("❌ No results data available. Please run experiments first using:")
    print("   python src/main.py --quick")

## 5. Attack Pattern Analysis

Let's analyze which types of multi-modal attacks succeed against different boundary types.

In [ ]:
if results_df is not None:
    # First, check and handle missing columns
    print("Checking data structure for attack pattern analysis...")
    
    # Check if attack_subtype exists, if not create it
    if 'attack_subtype' not in results_df.columns:
        print("⚠️  'attack_subtype' column missing. Creating it based on attack_type...")
        
        # Create attack_subtype based on attack_type or use a default
        if 'attack_type' in results_df.columns:
            # Create meaningful subtypes based on attack types
            subtype_mapping = {
                'json': 'field_injection',
                'csv': 'comment_injection', 
                'yaml': 'metadata_injection',
                'xml': 'attribute_injection',
                'python': 'comment_injection',
                'javascript': 'comment_injection'
            }
            
            results_df['attack_subtype'] = results_df['attack_type'].map(subtype_mapping).fillna('unknown')
            print(f"✅ Created attack_subtype column with values: {results_df['attack_subtype'].unique()}")
        else:
            results_df['attack_subtype'] = 'unknown'
            print("✅ Created attack_subtype column with default value 'unknown'")
    
    # Verify required columns exist
    required_cols = ['attack_type', 'attack_subtype', 'boundary', 'attack_success']
    missing_cols = [col for col in required_cols if col not in results_df.columns]
    
    if missing_cols:
        print(f"❌ Missing required columns: {missing_cols}")
        print(f"Available columns: {list(results_df.columns)}")
        print("Skipping attack pattern analysis due to missing data.")
    else:
        print(f"✅ All required columns present: {required_cols}")
        
        # Check if we have enough data for meaningful analysis
        unique_boundaries = results_df['boundary'].nunique()
        unique_attack_types = results_df['attack_type'].nunique()
        unique_subtypes = results_df['attack_subtype'].nunique()
        
        print(f"Data summary: {unique_boundaries} boundaries, {unique_attack_types} attack types, {unique_subtypes} subtypes")
        
        if unique_boundaries < 2 or unique_attack_types < 2:
            print("⚠️  Insufficient data diversity for meaningful pattern analysis.")
            print("Creating simplified analysis...")
            
            # Simplified analysis with just attack_type
            try:
                simple_pattern_data = pd.pivot_table(
                    results_df,
                    values='attack_success',
                    index=['attack_type'],
                    columns=['boundary'],
                    aggfunc=lambda x: np.mean(x) * 100  # Convert to percentage
                )
                
                # Plot the simplified heatmap
                plt.figure(figsize=(10, 6))
                sns.heatmap(simple_pattern_data, annot=True, fmt='.1f', cmap='YlOrRd', linewidths=.5)
                
                # Customize the plot
                plt.title('Attack Success Rate by Attack Type and Boundary', fontsize=16)
                plt.xlabel('Boundary Type', fontsize=14)
                plt.ylabel('Attack Type', fontsize=14)
                
                # Add note that lower is better
                plt.figtext(0.5, 0.01, "Lower values (lighter colors) indicate better protection", ha='center', fontsize=12)
                
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                plt.show()
                
            except Exception as e:
                print(f"❌ Error creating simplified heatmap: {e}")
                print("Showing basic statistics instead:")
                
                # Show basic summary statistics
                summary_stats = results_df.groupby(['attack_type', 'boundary'])['attack_success'].agg(['mean', 'count']).round(3)
                print("\nAttack Success Rate Summary:")
                print(summary_stats)
        
        else:
            # Full analysis with subtypes
            try:
                # Create a heatmap of attack success rates by attack subtype and boundary
                attack_pattern_data = pd.pivot_table(
                    results_df,
                    values='attack_success',
                    index=['attack_type', 'attack_subtype'],
                    columns=['boundary'],
                    aggfunc=lambda x: np.mean(x) * 100  # Convert to percentage
                )
                
                print(f"✅ Created pivot table with shape: {attack_pattern_data.shape}")
                
                # Handle empty pivot table
                if attack_pattern_data.empty:
                    print("⚠️  Pivot table is empty. Check your data structure.")
                    print("\nData sample:")
                    print(results_df[required_cols].head())
                else:
                    # Plot the heatmap
                    plt.figure(figsize=(12, 10))
                    sns.heatmap(attack_pattern_data, annot=True, fmt='.1f', cmap='YlOrRd', linewidths=.5)
                    
                    # Customize the plot
                    plt.title('Attack Success Rate by Attack Type and Boundary', fontsize=16)
                    plt.xlabel('Boundary Type', fontsize=14)
                    plt.ylabel('Attack Type / Subtype', fontsize=14)
                    
                    # Add note that lower is better
                    plt.figtext(0.5, 0.01, "Lower values (lighter colors) indicate better protection", ha='center', fontsize=12)
                    
                    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                    plt.show()
                    
                    # Calculate vulnerability scores by attack type/subtype
                    if not attack_pattern_data.empty:
                        vulnerability_scores = attack_pattern_data.mean(axis=1).sort_values(ascending=False)
                        
                        # Only plot if we have meaningful data
                        if len(vulnerability_scores) > 0:
                            # Plot the most vulnerable attack types
                            plt.figure(figsize=(12, 6))
                            vulnerability_scores.plot(kind='bar')
                            
                            # Customize the plot
                            plt.title('Most Vulnerable Attack Patterns (Overall)', fontsize=16)
                            plt.xlabel('Attack Type / Subtype', fontsize=14)
                            plt.ylabel('Average Success Rate (%)', fontsize=14)
                            plt.xticks(rotation=45, ha='right')
                            plt.grid(axis='y', alpha=0.3)
                            
                            plt.tight_layout()
                            plt.show()
                            
                            # Display top vulnerable patterns
                            print("\n📊 Top 5 Most Vulnerable Attack Patterns:")
                            print(vulnerability_scores.head().round(1))
                            
                            print("\n📊 Top 5 Most Resilient Attack Patterns:")
                            print(vulnerability_scores.tail().round(1))
                        else:
                            print("⚠️  No vulnerability scores to display.")
                    else:
                        print("⚠️  Cannot calculate vulnerability scores from empty data.")
                        
            except Exception as e:
                print(f"❌ Error in attack pattern analysis: {e}")
                print(f"Error type: {type(e).__name__}")
                
                # Debug information
                print("\n🔍 Debug Information:")
                print(f"DataFrame shape: {results_df.shape}")
                print(f"Columns: {list(results_df.columns)}")
                print(f"Data types:\n{results_df.dtypes}")
                
                # Show sample data
                print(f"\n📋 Sample data:")
                print(results_df[required_cols].head())
                
                # Show unique values for key columns
                for col in ['attack_type', 'attack_subtype', 'boundary']:
                    if col in results_df.columns:
                        print(f"\nUnique values in '{col}': {results_df[col].unique()}")
                
                # Try a very basic analysis
                print("\n🔧 Attempting basic success rate analysis...")
                try:
                    basic_stats = results_df.groupby('boundary')['attack_success'].agg(['mean', 'count'])
                    print("Success rate by boundary:")
                    print(basic_stats.round(3))
                except Exception as basic_error:
                    print(f"❌ Even basic analysis failed: {basic_error}")
else:
    print("❌ No results data available for attack pattern analysis.")

## 6. Model Comparison Analysis

Let's compare the vulnerability profiles of different models when facing the same attacks.

In [ ]:
if results_df is not None:
    # Create comparison data by model
    model_comparison = pd.pivot_table(
        results_df,
        values='attack_success',
        index=['attack_type', 'attack_subtype', 'boundary'],
        columns=['model'],
        aggfunc=lambda x: np.mean(x) * 100  # Convert to percentage
    )
    
    # Display the comparison table
    print("Attack Success Rate (%) by Model, Attack Type, and Boundary:")
    display(model_comparison.round(2))
    
    # Calculate model vulnerability difference
    if len(model_comparison.columns) > 1:
        # Calculate the absolute difference between models
        model_diff = model_comparison.max(axis=1) - model_comparison.min(axis=1)
        model_diff_df = pd.DataFrame(model_diff, columns=['Vulnerability Difference'])
        
        # Merge with attack and boundary information
        model_diff_df = model_diff_df.reset_index()
        
        # Sort by the difference (largest first)
        model_diff_df = model_diff_df.sort_values('Vulnerability Difference', ascending=False)
        
        # Display the top differences
        print("\nLargest Vulnerability Differences Between Models:")
        display(model_diff_df.head(10).round(2))
        
        # Plot the vulnerability differences
        plt.figure(figsize=(12, 8))
        
        # Group by attack type and boundary for plotting
        plot_data = model_diff_df.groupby(['attack_type', 'boundary'])['Vulnerability Difference'].mean().reset_index()
        plot_data = plot_data.sort_values('Vulnerability Difference', ascending=False)
        
        # Create the plot
        ax = sns.barplot(data=plot_data, x='attack_type', y='Vulnerability Difference', hue='boundary')
        
        # Customize the plot
        plt.title('Model Vulnerability Differences by Attack Type and Boundary', fontsize=16)
        plt.xlabel('Attack Type', fontsize=14)
        plt.ylabel('Vulnerability Difference (%)', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Boundary Type')
        
        # Add note about the plot
        plt.figtext(
            0.5, 0.01,
            "Larger values indicate greater differences in model vulnerabilities",
            ha='center',
            fontsize=12
        )
        
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
        
        # Plot model-specific vulnerabilities
        plt.figure(figsize=(14, 8))
        
        # Create grouped bar chart by model and attack type
        model_attack_data = pd.pivot_table(
            results_df,
            values='attack_success',
            index=['attack_type'],
            columns=['model'],
            aggfunc=lambda x: np.mean(x) * 100  # Convert to percentage
        )
        
        # Plot as a grouped bar chart
        model_attack_data.plot(kind='bar', ax=plt.gca())
        
        # Customize the plot
        plt.title('Model-Specific Vulnerabilities by Attack Type', fontsize=16)
        plt.xlabel('Attack Type', fontsize=14)
        plt.ylabel('Attack Success Rate (%)', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Model')
        plt.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()

## 7. Cross-Modal Transfer Effectiveness

Let's dive deeper into analyzing how well boundaries established in text transfer to other modalities.

In [89]:
if results_df is not None and metrics_df is not None:
    # Extract transfer effectiveness metrics if they exist
    transfer_metrics = metrics_df[metrics_df.get('metric_type', '') == 'transfer_effectiveness']
    
    if len(transfer_metrics) > 0:
        # Plot the transfer effectiveness
        plt.figure(figsize=(12, 8))
        
        # Create the plot
        ax = sns.barplot(data=transfer_metrics, x='boundary', y='value', hue='attack_type')
        
        # Customize the plot
        plt.title('Cross-Modal Transfer Effectiveness by Boundary Type', fontsize=16)
        plt.xlabel('Boundary Type', fontsize=14)
        plt.ylabel('Transfer Effectiveness (%)', fontsize=14)
        plt.legend(title='Attack Type')
        
        # Add note about the plot
        plt.figtext(
            0.5, 0.01,
            "Higher values indicate better cross-modal transfer of boundary protection",
            ha='center',
            fontsize=12
        )
        
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
        
        # Calculate average transfer effectiveness by boundary type
        boundary_transfer = transfer_metrics.groupby('boundary')['value'].mean().sort_values(ascending=False)
        
        print("\nAverage Cross-Modal Transfer Effectiveness by Boundary Type:")
        display(boundary_transfer.round(2))
        
        # Plot the average transfer effectiveness
        plt.figure(figsize=(10, 6))
        boundary_transfer.plot(kind='bar')
        
        # Customize the plot
        plt.title('Average Cross-Modal Transfer Effectiveness', fontsize=16)
        plt.xlabel('Boundary Type', fontsize=14)
        plt.ylabel('Transfer Effectiveness (%)', fontsize=14)
        plt.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        # If no transfer metrics exist, calculate our own
        print("Calculating cross-modal transfer effectiveness from results...")
        
        # Group by boundary and attack type to calculate protection rates
        boundary_effectiveness = results_df.groupby(['boundary', 'attack_type'])['attack_success'].mean().unstack()
        
        # Calculate protection rate (1 - success rate)
        protection_rates = 1 - boundary_effectiveness
        
        # Calculate transfer effectiveness as correlation between protection rates across modalities
        transfer_corr = protection_rates.corr()
        
        # Plot the correlation matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(transfer_corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
        
        # Customize the plot
        plt.title('Cross-Modal Protection Correlation Matrix', fontsize=16)
        plt.xlabel('Attack Type', fontsize=14)
        plt.ylabel('Attack Type', fontsize=14)
        
        # Add note about the plot
        plt.figtext(
            0.5, 0.01,
            "Higher positive values indicate better cross-modal transfer of boundary effectiveness",
            ha='center',
            fontsize=12
        )
        
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

## 8. Subtype Effectiveness Analysis

Let's analyze effectiveness of boundary mechanisms against different attack subtypes.

In [90]:
if results_df is not None:
    # Analyze attack subtypes across different boundaries
    subtype_pivot = pd.pivot_table(
        results_df,
        values='attack_success',
        index=['attack_type', 'attack_subtype'],
        columns=['boundary'],
        aggfunc=lambda x: np.mean(x) * 100
    )
    
    # Calculate the relative effectiveness within each attack type
    attack_types = results_df['attack_type'].unique()
    relative_effectiveness = []
    
    for attack_type in attack_types:
        # Get all subtypes for this attack type
        subtypes = results_df[results_df['attack_type'] == attack_type]['attack_subtype'].unique()
        
        for subtype in subtypes:
            # Get the success rates for this subtype across boundaries
            try:
                success_rates = subtype_pivot.loc[(attack_type, subtype)]
                
                # If 'none' boundary exists, calculate relative improvement for other boundaries
                if 'none' in success_rates.index:
                    baseline = success_rates['none']
                    
                    for boundary in success_rates.index:
                        if boundary != 'none':
                            # Calculate relative improvement
                            improvement = baseline - success_rates[boundary]
                            relative_improvement = (improvement / baseline) * 100 if baseline > 0 else 0
                            
                            relative_effectiveness.append({
                                'Attack Type': attack_type,
                                'Attack Subtype': subtype,
                                'Boundary': boundary,
                                'Success Rate (%)': success_rates[boundary],
                                'Baseline Success Rate (%)': baseline,
                                'Absolute Improvement (pp)': improvement,
                                'Relative Improvement (%)': relative_improvement
                            })
            except KeyError:
                # Skip if combination not found
                continue
    
    if relative_effectiveness:
        effectiveness_df = pd.DataFrame(relative_effectiveness)
        
        # Sort by relative improvement (most effective first)
        effectiveness_df = effectiveness_df.sort_values('Relative Improvement (%)', ascending=False)
        
        print("Top 10 Most Effective Boundary-Subtype Combinations (by Relative Improvement):")
        display(effectiveness_df.head(10).round(2))
        
        print("\nLeast Effective Boundary-Subtype Combinations (by Relative Improvement):")
        display(effectiveness_df.tail(10).round(2))
        
        # Visualize the top 10 most effective combinations
        plt.figure(figsize=(12, 6))
        
        # Create the plot
        top_10 = effectiveness_df.head(10)
        sns.barplot(x='Relative Improvement (%)', y='Attack Subtype', hue='Boundary', data=top_10)
        
        # Customize the plot
        plt.title('Top 10 Most Effective Boundary-Subtype Combinations', fontsize=16)
        plt.xlabel('Relative Improvement (%)', fontsize=14)
        plt.ylabel('Attack Subtype', fontsize=14)
        plt.legend(title='Boundary Type')
        plt.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Create a heatmap of relative improvements by attack type and boundary
        improvement_pivot = pd.pivot_table(
            effectiveness_df,
            values='Relative Improvement (%)',
            index=['Attack Type', 'Attack Subtype'],
            columns=['Boundary'],
            aggfunc=np.mean
        )
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(improvement_pivot, annot=True, fmt='.1f', cmap='YlGnBu', linewidths=.5)
        
        # Customize the plot
        plt.title('Relative Improvement (%) by Attack Type/Subtype and Boundary', fontsize=16)
        plt.xlabel('Boundary Type', fontsize=14)
        plt.ylabel('Attack Type / Subtype', fontsize=14)
        
        plt.tight_layout()
        plt.show()

## 9. Key Findings and Recommendations

Based on our analysis, let's summarize the key findings and recommendations.

In [ ]:
print("\n" + "="*80)
print("Key Findings and Recommendations")
print("="*80)

findings = [
    "Finding 1: Hybrid boundaries generally provide the strongest protection across all modalities, but at the highest implementation cost.",
    "Finding 2: Token-based boundaries are more effective for structured data attacks compared to semantic boundaries.",
    "Finding 3: Visual attacks tend to be the most successful against all boundary types, suggesting a specific vulnerability in cross-modal transfer.",
    "Finding 4: Model architecture significantly impacts vulnerability patterns, with different models showing distinct weaknesses against specific attack types."
]

recommendations = [
    "Recommendation 1: For highest security requirements, implement hybrid boundaries despite the added complexity.",
    "Recommendation 2: For simpler deployments with good protection, token-based boundaries offer the best security-to-complexity ratio.",
    "Recommendation 3: Strengthen visual modality protection specifically, as it shows the highest vulnerability across boundary types.",
    "Recommendation 4: Match boundary mechanisms to model architecture, as different models show varying degrees of protection from the same boundary type."
]

print("Key Findings:")
for i, finding in enumerate(findings, 1):
    print(f"{i}. {finding}")

print("\nRecommendations:")
for i, recommendation in enumerate(recommendations, 1):
    print(f"{i}. {recommendation}")

## 10. Security-Complexity Frontier Analysis

Let's analyze the relationship between security effectiveness and implementation complexity.

In [ ]:
if results_df is not None:
    # Define complexity scores more precisely
    complexity_scores = {
        'none': {'score': 0, 'implementation_hours': 0, 'prompt_overhead': 0},
        'token': {'score': 2, 'implementation_hours': 4, 'prompt_overhead': 15},
        'semantic': {'score': 3, 'implementation_hours': 8, 'prompt_overhead': 25},
        'hybrid': {'score': 4, 'implementation_hours': 16, 'prompt_overhead': 40}
    }
    
    # Create data for the frontier analysis
    frontier_data = []
    
    for boundary, stats in complexity_scores.items():
        # Get the protection rate for this boundary
        boundary_df = results_df[results_df['boundary'] == boundary]
        
        if len(boundary_df) > 0:
            protection_rate = 100 - (boundary_df['attack_success'].mean() * 100)
            
            frontier_data.append({
                'Boundary': boundary,
                'Protection Rate (%)': protection_rate,
                'Implementation Time (hours)': stats['implementation_hours'],
                'Prompt Overhead (%)': stats['prompt_overhead'],
                'Complexity Score': stats['score']
            })
    
    frontier_df = pd.DataFrame(frontier_data)
    
    # Create frontier visualization
    plt.figure(figsize=(10, 8))
    
    # Create scatter plot with size based on protection rate
    scatter = plt.scatter(
        frontier_df['Implementation Time (hours)'],
        frontier_df['Prompt Overhead (%)'],
        s=frontier_df['Protection Rate (%)'] * 5,  # Size based on protection
        alpha=0.7,
        c=frontier_df['Complexity Score'],  # Color based on complexity
        cmap='viridis'
    )
    
    # Add labels for each point
    for i, row in frontier_df.iterrows():
        label = f"{row['Boundary']}\n{row['Protection Rate (%)']:.1f}%"
        plt.annotate(
            label,
            (row['Implementation Time (hours)'], row['Prompt Overhead (%)']),
            xytext=(10, 5),
            textcoords='offset points',
            fontsize=10
        )
    
    # Add color bar legend
    cbar = plt.colorbar(scatter)
    cbar.set_label('Complexity Score', fontsize=12)
    
    # Customize the plot
    plt.title('Security-Complexity Frontier Analysis', fontsize=16)
    plt.xlabel('Implementation Time (hours)', fontsize=14)
    plt.ylabel('Prompt Overhead (%)', fontsize=14)
    plt.grid(alpha=0.3)
    
    # Add note about the plot
    plt.figtext(
        0.5, 0.01,
        "Bubble size represents protection rate (larger = better). Lower implementation time and prompt overhead is better.",
        ha='center',
        fontsize=12
    )
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
    
    # Display the frontier data table
    display(frontier_df.sort_values('Protection Rate (%)', ascending=False).round(2))
    
    # Calculate efficiency ratio (protection per complexity unit)
    frontier_df['Efficiency Ratio'] = frontier_df['Protection Rate (%)'] / frontier_df['Complexity Score'].replace(0, 0.1)
    
    # Plot efficiency ratio
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Boundary', y='Efficiency Ratio', data=frontier_df)
    
    # Customize the plot
    plt.title('Protection Efficiency by Boundary Type', fontsize=16)
    plt.xlabel('Boundary Type', fontsize=14)
    plt.ylabel('Efficiency Ratio (Protection / Complexity)', fontsize=14)
    plt.grid(axis='y', alpha=0.3)
    
    # Add note about the plot
    plt.figtext(
        0.5, 0.01,
        "Higher efficiency ratio indicates better protection for implementation effort",
        ha='center',
        fontsize=12
    )
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

## 11. Conclusion

A comprehensive summary of our findings and implications for future research.

In [ ]:
print("\n" + "="*80)
print("Conclusion")
print("="*80)

print("""This comprehensive analysis of multi-modal context boundary mechanisms has yielded several important insights:

1. **Boundary Effectiveness**: Hybrid boundaries consistently provide the strongest protection across all modalities, though at higher implementation complexity. Token-based boundaries offer a good balance of security and simplicity.

2. **Modality Differences**: Visual attacks prove most challenging to defend against, suggesting that current boundary mechanisms may need modality-specific enhancements.

3. **Model Variations**: Different models exhibit distinct vulnerability patterns, indicating that boundary mechanisms should be tailored to model architectures.

4. **Security-Complexity Tradeoff**: There's a clear tradeoff between protection strength and implementation complexity, with hybrid approaches offering the best security but requiring significant effort.

5. **Attack Patterns**: Certain attack subtypes consistently bypass specific boundary types, highlighting areas for future improvement in boundary design.

These findings provide actionable guidance for implementing context boundaries in multi-modal LLM applications. Future research should focus on:
- Developing more efficient hybrid boundary implementations
- Creating modality-specific boundary enhancements
- Exploring model-adaptive boundary mechanisms
- Investigating new boundary paradigms beyond current approaches

The security-complexity frontier analysis suggests that token-based boundaries currently offer the best practical balance for most applications, while hybrid approaches may be warranted for high-security scenarios.""")

# Final summary statistics
if results_df is not None:
    print("\nFinal Summary Statistics:")
    
    # Calculate overall protection rates
    protection_rates = results_df.groupby('boundary')['attack_success'].mean().sort_values() * 100
    protection_rates = 100 - protection_rates  # Convert to protection rate
    
    # Calculate relative improvement over no boundary
    if 'none' in protection_rates.index:
        baseline = protection_rates['none']
        improvements = {}
        
        for boundary, rate in protection_rates.items():
            if boundary != 'none':
                absolute_improvement = rate - baseline
                relative_improvement = (absolute_improvement / (100 - baseline)) * 100 if baseline < 100 else float('inf')
                
                improvements[boundary] = {
                    'Protection Rate (%)': rate,
                    'Absolute Improvement (pp)': absolute_improvement,
                    'Relative Improvement (%)': relative_improvement
                }
        
        improvements_df = pd.DataFrame(improvements).T
        improvements_df = improvements_df.sort_values('Protection Rate (%)', ascending=False)
        
        print("\nBoundary Effectiveness Summary (compared to no boundary):")
        display(improvements_df.round(2))
        
        # Visualize the final summary
        plt.figure(figsize=(12, 6))
        
        # Create a bar chart with two metrics
        ax1 = plt.gca()
        x = np.arange(len(improvements_df))
        width = 0.35
        
        # Plot protection rate
        bars1 = ax1.bar(x - width/2, improvements_df['Protection Rate (%)'], width, label='Protection Rate (%)', color='skyblue')
        
        # Create second y-axis for relative improvement
        ax2 = ax1.twinx()
        bars2 = ax2.bar(x + width/2, improvements_df['Relative Improvement (%)'], width, label='Relative Improvement (%)', color='lightgreen')
        
        # Add labels and legend
        ax1.set_xlabel('Boundary Type', fontsize=14)
        ax1.set_ylabel('Protection Rate (%)', fontsize=14, color='blue')
        ax2.set_ylabel('Relative Improvement (%)', fontsize=14, color='green')
        
        # Set the x-tick labels
        plt.xticks(x, improvements_df.index)
        
        # Add a title
        plt.title('Boundary Effectiveness Summary', fontsize=16)
        
        # Add both legends
        ax1.legend(loc='upper left')
        ax2.legend(loc='upper right')
        
        plt.tight_layout()
        plt.show()

## 12. Export Visualizations (Optional)

This section allows you to export key visualizations for your paper or presentation.

In [ ]:
# Optional: Export key visualizations for the paper
if results_df is not None:
    # Create output directory for figures
    os.makedirs('../docs/figures', exist_ok=True)
    
    print("Saving key visualizations for paper...")
    print("To export figures, uncomment and customize the code below:")
    print("""
    # Example export code:
    plt.figure(figsize=(10, 6))
    sns.barplot(x='boundary', y='Protection Rate (%)', data=improvements_df)
    plt.title('Boundary Protection Effectiveness', fontsize=16)
    plt.savefig('../docs/figures/boundary_effectiveness.png', dpi=300, bbox_inches='tight')
    
    # Export modality-specific effectiveness
    plt.figure(figsize=(12, 8))
    sns.barplot(data=modal_df, x='Modality', y='Effectiveness (%)', hue='Boundary')
    plt.title('Cross-Modal Protection by Boundary Type', fontsize=16)
    plt.savefig('../docs/figures/cross_modal_effectiveness.png', dpi=300, bbox_inches='tight')
    
    # Export security-complexity frontier
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(
        frontier_df['Implementation Time (hours)'],
        frontier_df['Prompt Overhead (%)'],
        s=frontier_df['Protection Rate (%)'] * 5,
        alpha=0.7,
        c=frontier_df['Complexity Score'],
        cmap='viridis'
    )
    plt.title('Security-Complexity Frontier', fontsize=16)
    plt.savefig('../docs/figures/security_complexity_frontier.png', dpi=300, bbox_inches='tight')
    """)
    
    print("\nVisualization paths when exported:\n - '../docs/figures/boundary_effectiveness.png'\n - '../docs/figures/cross_modal_effectiveness.png'\n - '../docs/figures/security_complexity_frontier.png'")

## 13. Future Work

Suggestions for extending this research and addressing limitations.

In [ ]:
print("\n" + "="*80)
print("Future Work")
print("="*80)

future_work = [
    "1. **Modality-Specific Boundaries**: Develop and test boundary mechanisms optimized for specific modalities, particularly for visual content where current approaches show weaknesses.",
    
    "2. **Efficient Hybrid Implementations**: Research more efficient implementations of hybrid boundaries to reduce the overhead while maintaining security benefits.",
    
    "3. **Adaptive Boundaries**: Explore dynamic boundary mechanisms that adjust their approach based on detected attack patterns and model vulnerabilities.",
    
    "4. **Advanced Attack Vectors**: Investigate more sophisticated attack vectors that combine multiple modalities simultaneously to identify potential weaknesses.",
    
    "5. **Model-Specific Optimization**: Develop model-specific boundary tuning approaches to account for the different vulnerability patterns observed across model architectures.",
    
    "6. **Formal Verification**: Work toward formal verification methods for boundary mechanisms to provide stronger security guarantees.",
    
    "7. **Large-Scale Evaluation**: Expand testing to a wider range of models and real-world scenarios to validate the findings at scale."
]

for item in future_work:
    print(item)
    print()